In [ ]:
import cv2
import numpy as np
import time, os
import mediapipe as mp
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    return results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [ ]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

In [ ]:
def extract_keypoints(results):
    pose = np.array([(res.x, res.y, res.z, res.visibility) for res in results.pose_landmarks.landmark]).flatten() \
        if results.pose_landmarks.landmark \
        else np.zeros(33 * 4)
    face = np.array([(res.x, res.y, res.z) for res in results.face_landmarks.landmark]).flatten() \
        if results.face_landmarks.landmark \
        else np.zeors(468 * 3)
    lhand = np.array([(res.x, res.y, res.z) for res in results.left_hand_landmarks.landmark]).flatten() \
        if results.left_hand_landmarks \
        else np.zeros(21 * 3)
    rhand = np.array([(res.x, res.y, res.z) for res in results.right_hand_landmarks.landmark]).flatten() \
        if results.right_hand_landmarks \
        else np.zeros(21 * 3)
    return np.concatenate([pose, face, lhand, rhand])

In [ ]:
DATA_PATH = os.path.join('MP_Data')
actions = ['hello', 'thanks', 'iloveyou']
no_sequences = 30
sequence_length = 30

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):    
                state, frame = cap.read()

                results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(frame, results)
                
                cv2.putText(frame, f'Collecting frames for {action} Video Number {sequence}', (15, 12),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                if frame_num == 0:
                    cv2.putText(frame, 'STARTING COLLECTION', (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.waitKey(1000)

                cv2.imshow("OpenCV Feed", frame)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join('MP_Data', action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
        
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
action_map = {action: num for num, action in enumerate(actions)}

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        frames = []
        for frame_num in range(sequence_length):
            frame = np.load(os.path.join('MP_Data', action, str(sequence), f'{frame_num}.npy'))
            frames.append(frame)
        sequences.append(frames)
        labels.append(action_map[action])

In [ ]:
x = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.shape(actions)[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
res = model.predict(x_test)

In [ ]:
actions[np.argmax(y_test[0])]

In [ ]:
actions[np.argmax(res[0])]

In [ ]:
model.save('actions.h5')

In [ ]:
model.load_weights('actions.h5')

In [ ]:
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame

In [ ]:
sequence = []
sentence = []
threshold = 0.7
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        state, frame = cap.read()
        results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(frame, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]

            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            frame = prob_viz(res, actions, frame, colors)

        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(frame, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(frame, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow("OpenCV Feed", frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()